it looks like the model is picking up the diagnosis really well because it is in the discharge notes, which makes sense.  Let's see if maybe we can get the model to predict the right DRG instead?

In [1]:
import sqlite3 as sl
import pandas as pd
conn = sl.connect('../Data/patient_data.db')
c = conn.cursor()

In [2]:
drg = pd.read_sql('''select * from DRG limit 10;''',conn)
drg

,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",None,None
1,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,None,None
2,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,None,None
3,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,None,None
4,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,None,None
5,23705,154221,HCFA,385,"NEONATES, DIED OR TRANSFERRED TO ANOTHER ACUTE...",None,None
6,15787,122380,HCFA,105,CARDIAC VALVE & OTHER MAJOR CARDIOTHORACIC PRO...,None,None
7,13933,194891,HCFA,101,OTHER RESPIRATORY SYSTEM DIAGNOSES WITH COMPLI...,None,None
8,23976,171485,HCFA,107,CORONARY BYPASS WITH CARDIAC CATHETER,None,None
9,4351,164767,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,None,None


In [3]:
data = pd.read_sql("""
                   SELECT adm.subject_id, 
                          adm.hadm_id,
                          admission_type,
                          admission_location,
                          notes.chartdate,
                          notes.charttime,
                          notes.category,
                          notes.description,
                          notes.iserror,
                          notes.text,
                          adm.diagnosis,
                          drg.DRG_CODE,
                          drg.DRG_TYPE,
                          drg.DESCRIPTION AS DRG_DESC,
                          drg.DRG_SEVERITY,
                          drg.DRG_MORTALITY
                    FROM admissions as adm
                    JOIN notes on adm.subject_id = notes.subject_id
                         AND adm.hadm_id = notes.hadm_id
                    JOIN drg on drg.subject_id = adm.subject_id and drg.hadm_id = adm.hadm_id
                    WHERE adm.diagnosis in ('GASTROINTESTINAL BLEED','INTRACRANIAL HEMORRHAGE') 
                         AND category not in ('Discharge summary')
                    ORDER BY adm.subject_id, adm.hadm_id, chartdate""",conn)
data.to_csv('../Data/fmodel_data.csv')
data

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
0,46,144073,EMERGENCY,CLINIC REFERRAL/PREMATURE,2133-02-20,2133-02-20 18:56:00,Radiology,DISTINCT PROCEDURAL SERVICE,NaN,[**2133-2-20**] 6:56 PM\n MESENTERIC ...,GASTROINTESTINAL BLEED,207,HCFA,DISORDERS OF THE BILIARY TRACT WITH COMPLICATI...,NaN,NaN
1,46,144073,EMERGENCY,CLINIC REFERRAL/PREMATURE,2133-02-21,2133-02-21 05:07:00,Nursing/other,Report,NaN,MICU NPN 7P-7A\nPt. came from IR around 2200 p...,GASTROINTESTINAL BLEED,207,HCFA,DISORDERS OF THE BILIARY TRACT WITH COMPLICATI...,NaN,NaN
2,46,144073,EMERGENCY,CLINIC REFERRAL/PREMATURE,2133-02-23,2133-02-23 10:48:00,Radiology,CT PELVIS W/CONTRAST,NaN,[**2133-2-23**] 10:48 AM\n CTA ABD W&W/O C & R...,GASTROINTESTINAL BLEED,207,HCFA,DISORDERS OF THE BILIARY TRACT WITH COMPLICATI...,NaN,NaN
3,112,174105,EMERGENCY,EMERGENCY ROOM ADMIT,2194-06-13,None,ECG,Report,NaN,Sinus rhythm\nAnterolateral ST-T changes sugge...,GASTROINTESTINAL BLEED,174,HCFA,GASTROINTESTINAL HEMORRHAGE WITH COMPLICATIONS...,NaN,NaN
4,112,174105,EMERGENCY,EMERGENCY ROOM ADMIT,2194-06-13,None,ECG,Report,NaN,Sinus rhythm\nInferior/lateral ST-T changes ar...,GASTROINTESTINAL BLEED,174,HCFA,GASTROINTESTINAL HEMORRHAGE WITH COMPLICATIONS...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81356,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 19:07:00,Radiology,REMOVE TUNNELED CENTRAL W/O PORT,NaN,[**2183-2-14**] 7:07 PM\n TUNNEL DIALYSIS REPL...,GASTROINTESTINAL BLEED,8134,APR,Other Complications of Treatment,4.0,3.0
81357,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 19:07:00,Radiology,REMOVE TUNNELED CENTRAL W/O PORT,NaN,[**2183-2-14**] 7:07 PM\n TUNNEL DIALYSIS REPL...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81358,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-16,None,ECG,Report,NaN,Sinus rhythm. Left ventricular hypertrophy wit...,GASTROINTESTINAL BLEED,8134,APR,Other Complications of Treatment,4.0,3.0
81359,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-16,None,ECG,Report,NaN,Sinus rhythm. Left ventricular hypertrophy wit...,GASTROINTESTINAL BLEED,8134,APR,Other Complications of Treatment,4.0,3.0


In [4]:
data.DRG_CODE.value_counts()

44      4659
54      4026
444     3994
443     3408
214     3009
        ... 
4603       4
7914       4
2403       4
2212       3
683        2
Name: DRG_CODE, Length: 277, dtype: int64

In [5]:
data.DRG_TYPE.value_counts()

APR     46675
MS      20383
HCFA    14303
Name: DRG_TYPE, dtype: int64

In [6]:
ms = data[(data.DRG_TYPE == 'MS')]
ms

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
972,907,133125,EMERGENCY,CLINIC REFERRAL/PREMATURE,2163-10-01,None,ECG,Report,NaN,Normal sinus rhythm. Early precordial R wave ...,INTRACRANIAL HEMORRHAGE,23,MS,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,NaN,NaN
975,907,133125,EMERGENCY,CLINIC REFERRAL/PREMATURE,2163-10-01,2163-10-01 22:00:00,Radiology,CT HEAD W/O CONTRAST,NaN,[**2163-10-1**] 10:00 PM\n CT HEAD W/O CONTRAS...,INTRACRANIAL HEMORRHAGE,23,MS,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,NaN,NaN
978,907,133125,EMERGENCY,CLINIC REFERRAL/PREMATURE,2163-10-01,2163-10-01 22:04:00,Radiology,CHEST (PORTABLE AP),NaN,[**2163-10-1**] 10:04 PM\n CHEST (PORTABLE AP)...,INTRACRANIAL HEMORRHAGE,23,MS,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,NaN,NaN
981,907,133125,EMERGENCY,CLINIC REFERRAL/PREMATURE,2163-10-01,2163-10-01 23:07:00,Radiology,CTA HEAD W&W/O C & RECONS,NaN,[**2163-10-1**] 11:07 PM\n CTA HEAD W&W/O C & ...,INTRACRANIAL HEMORRHAGE,23,MS,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,NaN,NaN
984,907,133125,EMERGENCY,CLINIC REFERRAL/PREMATURE,2163-10-02,2163-10-02 00:40:00,Radiology,CT HEAD W/O CONTRAST,NaN,[**2163-10-2**] 12:40 AM\n CT HEAD W/O CONTRAS...,INTRACRANIAL HEMORRHAGE,23,MS,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81348,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-10,2183-02-10 02:23:00,Radiology,CHEST (PORTABLE AP),NaN,[**2183-2-10**] 2:23 AM\n CHEST (PORTABLE AP) ...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81351,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-12,None,Echo,Report,NaN,PATIENT/TEST INFORMATION:\nIndication: CAD. Re...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81354,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 09:12:00,Radiology,VENOUS DUP UPPER EXT BILATERAL,NaN,[**2183-2-14**] 9:12 AM\n VENOUS DUP UPPER EXT...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81357,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 19:07:00,Radiology,REMOVE TUNNELED CENTRAL W/O PORT,NaN,[**2183-2-14**] 7:07 PM\n TUNNEL DIALYSIS REPL...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN


In [7]:
ms.DRG_CODE.value_counts()

4      2675
3      2615
64     2315
377    1518
23     1453
       ... 
376       6
314       6
385       4
392       4
683       2
Name: DRG_CODE, Length: 85, dtype: int64

In [8]:
gi_bleed_ms = ms[(ms.DIAGNOSIS == 'GASTROINTESTINAL BLEED')]
gi_bleed_ms

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
4838,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-28,None,ECG,Report,NaN,Sinus tachycardia. Low QRS voltage. Borderli...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
4841,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-28,None,ECG,Report,NaN,Sinus tachycardia. Low QRS voltage. Borderli...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
4844,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-28,2145-12-28 17:34:00,Radiology,CHEST (PORTABLE AP),NaN,[**2145-12-28**] 5:34 PM\n CHEST (PORTABLE AP)...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
4847,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-28,2145-12-28 20:31:00,Radiology,CT ABDOMEN W/CONTRAST,NaN,[**2145-12-28**] 8:31 PM\n CT ABDOMEN W/CONTRA...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
4850,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81348,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-10,2183-02-10 02:23:00,Radiology,CHEST (PORTABLE AP),NaN,[**2183-2-10**] 2:23 AM\n CHEST (PORTABLE AP) ...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81351,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-12,None,Echo,Report,NaN,PATIENT/TEST INFORMATION:\nIndication: CAD. Re...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81354,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 09:12:00,Radiology,VENOUS DUP UPPER EXT BILATERAL,NaN,[**2183-2-14**] 9:12 AM\n VENOUS DUP UPPER EXT...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81357,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 19:07:00,Radiology,REMOVE TUNNELED CENTRAL W/O PORT,NaN,[**2183-2-14**] 7:07 PM\n TUNNEL DIALYSIS REPL...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN


In [9]:
gi_bleed_ms.DRG_CODE.value_counts()

377    1518
4       761
378     601
871     310
326     295
329     199
432     198
405     195
368     153
435     140
393     138
237     135
374     122
379      89
920      84
356      76
207      72
391      64
330      53
441      50
327      45
682      44
870      39
386      37
957      37
907      34
369      32
394      30
347      29
565      27
698      24
919      21
382      19
208      19
981      17
381      17
166      17
982      16
289      14
812      14
299      11
811      11
375      11
444       9
820       7
252       7
376       6
477       6
987       6
385       4
392       4
683       2
Name: DRG_CODE, dtype: int64

In [10]:
physician_note = data[(data.DESCRIPTION == 'Physician Resident Admission Note')]
physician_note

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
4848,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,7204,APR,Septicemia & Disseminated Infections,4.0,4.0
4849,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,7204,APR,Septicemia & Disseminated Infections,4.0,4.0
4850,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
4851,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,7204,APR,Septicemia & Disseminated Infections,4.0,4.0
4852,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,7204,APR,Septicemia & Disseminated Infections,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78188,96610,117203,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2183-04-09,2183-04-09 11:10:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n HPI:\n Mr. [**Known las...,GASTROINTESTINAL BLEED,2442,APR,Diverticulitis & Diverticulosis,2.0,2.0
78189,96610,117203,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2183-04-09,2183-04-09 11:10:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n HPI:\n Mr. [**Known las...,GASTROINTESTINAL BLEED,378,MS,G.I. HEMORRHAGE W CC,NaN,NaN
80761,98814,185436,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2141-02-08,2141-02-08 18:29:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint: GI bleed\n HPI:\n 62 yo ...,GASTROINTESTINAL BLEED,2214,APR,Major Small & Large Bowel Procedures,4.0,3.0
80762,98814,185436,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2141-02-08,2141-02-08 18:29:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint: GI bleed\n HPI:\n 62 yo ...,GASTROINTESTINAL BLEED,2214,APR,Major Small & Large Bowel Procedures,4.0,3.0


In [11]:
val = physician_note.TEXT.values[10]
val

'Chief Complaint:  Chest pain, GI bleed\n   HPI:\n   59 y/o M with PMHx of ESRD on HD, GI bleeds, CAD and polysubstance\n   abuse who was brought into the ED via EMS after his wife witnessed a\n   syncopal episode. Pt was complaining of left sided chest pain but was\n   drowsy on arrival.\n   .\n   In the ED, initial vs were: T 96.8 P 60 BP 92/52 R O2 sat 100% on NRB.\n   Pt some new TWI on EKG in V2-V6 and was being bolused for hypotension.\n   At midnight, pt was noted to be having possible seizure activity with\n   left eye deviation and foaming at his mouth.  After this activity\n   ceased, pt was post ictal and unable to be aroused.  He was intubated\n   with etomidate and rocuronium due to concern for inability to protect\n   his airway.  Pt remained mildly hypotensive and hct came back at 24\n   (down from baseline of 30).  He had a right femoral CVL placed and\n   rectal exam revealed brown stool mixed with blood. OG tube was placed\n   and there was no evidence of hematemesis.

In [12]:
physician_note.DIAGNOSIS.value_counts()

GASTROINTESTINAL BLEED     402
INTRACRANIAL HEMORRHAGE     75
Name: DIAGNOSIS, dtype: int64

In [13]:
phys_note = data[(data.DESCRIPTION == 'Physician Resident Admission Note')& (data.DRG_TYPE == 'MS')]
phys_note

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
4850,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
4853,4962,128158,EMERGENCY,EMERGENCY ROOM ADMIT,2145-12-29,2145-12-29 02:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n Chief Complaint: hypotens...,GASTROINTESTINAL BLEED,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,NaN,NaN
6401,5727,178576,EMERGENCY,EMERGENCY ROOM ADMIT,2156-02-26,2156-02-26 05:05:00,Physician,Physician Resident Admission Note,NaN,"Chief Complaint: Chest pain, GI bleed\n HPI...",GASTROINTESTINAL BLEED,377,MS,G.I. HEMORRHAGE W MCC,NaN,NaN
6404,5727,178576,EMERGENCY,EMERGENCY ROOM ADMIT,2156-02-26,2156-02-26 05:05:00,Physician,Physician Resident Admission Note,NaN,"Chief Complaint: Chest pain, GI bleed\n HPI...",GASTROINTESTINAL BLEED,377,MS,G.I. HEMORRHAGE W MCC,NaN,NaN
7012,6262,170247,EMERGENCY,CLINIC REFERRAL/PREMATURE,2156-09-16,2156-09-16 00:14:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint: GI bleed\n HPI:\n Mr. [*...,GASTROINTESTINAL BLEED,377,MS,G.I. HEMORRHAGE W MCC,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77532,96310,185290,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2123-01-08,2123-01-08 01:33:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint: GI bleed\n HPI:\n 53 y.o...,GASTROINTESTINAL BLEED,369,MS,MAJOR ESOPHAGEAL DISORDERS W CC,NaN,NaN
77628,96468,171641,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2102-09-05,2102-09-05 21:38:00,Physician,Physician Resident Admission Note,NaN,TITLE:\n Chief Complaint:\n HPI:\n 79 y....,GASTROINTESTINAL BLEED,377,MS,G.I. HEMORRHAGE W MCC,NaN,NaN
78075,96591,105668,EMERGENCY,EMERGENCY ROOM ADMIT,2159-09-04,2159-09-04 00:13:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint: s/p PEA arrest\n HPI:\n ...,INTRACRANIAL HEMORRHAGE,3,MS,"ECMO OR TRACH W MV 96+ HRS OR PDX EXC FACE, MO...",NaN,NaN
78189,96610,117203,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,2183-04-09,2183-04-09 11:10:00,Physician,Physician Resident Admission Note,NaN,Chief Complaint:\n HPI:\n Mr. [**Known las...,GASTROINTESTINAL BLEED,378,MS,G.I. HEMORRHAGE W CC,NaN,NaN


In [14]:
data = pd.read_sql("""
                   SELECT adm.subject_id, 
                          adm.hadm_id,
                          admission_type,
                          admission_location,
                          notes.chartdate,
                          notes.charttime,
                          notes.category,
                          notes.description,
                          notes.iserror,
                          notes.text,
                          adm.diagnosis,
                          drg.DRG_CODE,
                          drg.DRG_TYPE,
                          drg.DESCRIPTION AS DRG_DESC,
                          drg.DRG_SEVERITY,
                          drg.DRG_MORTALITY
                    FROM admissions as adm
                    JOIN notes on adm.subject_id = notes.subject_id
                         AND adm.hadm_id = notes.hadm_id
                    JOIN drg on drg.subject_id = adm.subject_id and drg.hadm_id = adm.hadm_id
                    WHERE adm.diagnosis in ('GASTROINTESTINAL BLEED','INTRACRANIAL HEMORRHAGE') 
                         AND category not in ('Discharge summary')
                    ORDER BY adm.subject_id, adm.hadm_id, chartdate""",conn)
data.to_csv('../Data/new_model_data.csv')
data

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
0,46,144073,EMERGENCY,CLINIC REFERRAL/PREMATURE,2133-02-20,2133-02-20 18:56:00,Radiology,DISTINCT PROCEDURAL SERVICE,NaN,[**2133-2-20**] 6:56 PM\n MESENTERIC ...,GASTROINTESTINAL BLEED,207,HCFA,DISORDERS OF THE BILIARY TRACT WITH COMPLICATI...,NaN,NaN
1,46,144073,EMERGENCY,CLINIC REFERRAL/PREMATURE,2133-02-21,2133-02-21 05:07:00,Nursing/other,Report,NaN,MICU NPN 7P-7A\nPt. came from IR around 2200 p...,GASTROINTESTINAL BLEED,207,HCFA,DISORDERS OF THE BILIARY TRACT WITH COMPLICATI...,NaN,NaN
2,46,144073,EMERGENCY,CLINIC REFERRAL/PREMATURE,2133-02-23,2133-02-23 10:48:00,Radiology,CT PELVIS W/CONTRAST,NaN,[**2133-2-23**] 10:48 AM\n CTA ABD W&W/O C & R...,GASTROINTESTINAL BLEED,207,HCFA,DISORDERS OF THE BILIARY TRACT WITH COMPLICATI...,NaN,NaN
3,112,174105,EMERGENCY,EMERGENCY ROOM ADMIT,2194-06-13,None,ECG,Report,NaN,Sinus rhythm\nAnterolateral ST-T changes sugge...,GASTROINTESTINAL BLEED,174,HCFA,GASTROINTESTINAL HEMORRHAGE WITH COMPLICATIONS...,NaN,NaN
4,112,174105,EMERGENCY,EMERGENCY ROOM ADMIT,2194-06-13,None,ECG,Report,NaN,Sinus rhythm\nInferior/lateral ST-T changes ar...,GASTROINTESTINAL BLEED,174,HCFA,GASTROINTESTINAL HEMORRHAGE WITH COMPLICATIONS...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81356,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 19:07:00,Radiology,REMOVE TUNNELED CENTRAL W/O PORT,NaN,[**2183-2-14**] 7:07 PM\n TUNNEL DIALYSIS REPL...,GASTROINTESTINAL BLEED,8134,APR,Other Complications of Treatment,4.0,3.0
81357,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-14,2183-02-14 19:07:00,Radiology,REMOVE TUNNELED CENTRAL W/O PORT,NaN,[**2183-2-14**] 7:07 PM\n TUNNEL DIALYSIS REPL...,GASTROINTESTINAL BLEED,919,MS,COMPLICATIONS OF TREATMENT W MCC,NaN,NaN
81358,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-16,None,ECG,Report,NaN,Sinus rhythm. Left ventricular hypertrophy wit...,GASTROINTESTINAL BLEED,8134,APR,Other Complications of Treatment,4.0,3.0
81359,99562,129689,URGENT,TRANSFER FROM HOSP/EXTRAM,2183-02-16,None,ECG,Report,NaN,Sinus rhythm. Left ventricular hypertrophy wit...,GASTROINTESTINAL BLEED,8134,APR,Other Complications of Treatment,4.0,3.0


In [15]:
data2 = pd.read_sql("""
                   SELECT distinct
                          adm.subject_id, 
                          adm.hadm_id,
                          admission_type,
                          admission_location,
                          adm.diagnosis,
                          drg.DRG_CODE,
                          drg.DRG_TYPE,
                          drg.DESCRIPTION AS DRG_DESC,
                          drg.DRG_SEVERITY,
                          drg.DRG_MORTALITY
                    FROM admissions as adm
                    JOIN drg on drg.subject_id = adm.subject_id and drg.hadm_id = adm.hadm_id
                    ORDER BY adm.subject_id, adm.hadm_id""",conn)
data2.to_csv('../Data/data2.csv')
data2

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
0,2,163353,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,6402,APR,"Neonate, Bwt > 2499g, Normal Newborn Or Neonat...",2.0,1.0
1,2,163353,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,391,HCFA,NORMAL NEWBORN,NaN,NaN
2,3,145834,EMERGENCY,EMERGENCY ROOM ADMIT,HYPOTENSION,416,HCFA,SEPTICEMIA AGE >17,NaN,NaN
3,4,185777,EMERGENCY,EMERGENCY ROOM ADMIT,"FEVER,DEHYDRATION,FAILURE TO THRIVE",489,HCFA,HIV WITH MAJOR RELATED CONDITION,NaN,NaN
4,5,178980,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,391,HCFA,NORMAL NEWBORN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
100319,99992,197084,EMERGENCY,CLINIC REFERRAL/PREMATURE,RETROPERITONEAL HEMORRHAGE,907,MS,OTHER O.R. PROCEDURES FOR INJURIES W MCC,NaN,NaN
100320,99995,137810,ELECTIVE,PHYS REFERRAL/NORMAL DELI,ABDOMINAL AORTIC ANEURYSM/SDA,1732,APR,Other Vascular Procedures,2.0,3.0
100321,99995,137810,ELECTIVE,PHYS REFERRAL/NORMAL DELI,ABDOMINAL AORTIC ANEURYSM/SDA,237,MS,MAJOR CARDIOVASC PROCEDURES W MCC,NaN,NaN
100322,99999,113369,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SPONDYLOLISTHESIS/SDA,3042,APR,Dorsal & Lumbar Fusion Proc Except for Curvatu...,2.0,1.0


In [16]:
data3 = pd.read_sql("""
                   SELECT distinct
                          adm.subject_id, 
                          adm.hadm_id,
                          admission_type,
                          admission_location,
                          adm.diagnosis,
                          drg.DRG_CODE,
                          drg.DRG_TYPE,
                          drg.DESCRIPTION AS DRG_DESC,
                          drg.DRG_SEVERITY,
                          drg.DRG_MORTALITY
                    FROM admissions as adm
                    JOIN drg on drg.subject_id = adm.subject_id and drg.hadm_id = adm.hadm_id
                    WHERE DRG_TYPE = 'MS'
                    ORDER BY adm.subject_id, adm.hadm_id""",conn)
data3.to_csv('../Data/data3.csv')
data3

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DIAGNOSIS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
0,23,124321,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,BRAIN MASS,25,MS,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,None,None
1,34,144319,EMERGENCY,CLINIC REFERRAL/PREMATURE,BRADYCARDIA,243,MS,PERMANENT CARDIAC PACEMAKER IMPLANT W CC,None,None
2,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,VENTRAL HERNIA/SDA,353,MS,HERNIA PROCEDURES EXCEPT INGUINAL & FEMORAL W MCC,None,None
3,85,112077,EMERGENCY,CLINIC REFERRAL/PREMATURE,PNEUMONIA,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,None,None
4,107,174162,EMERGENCY,CLINIC REFERRAL/PREMATURE,UPPER GI BLEED,380,MS,COMPLICATED PEPTIC ULCER W MCC,None,None
...,...,...,...,...,...,...,...,...,...,...
27274,99985,176670,EMERGENCY,EMERGENCY ROOM ADMIT,FEVER,870,MS,SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,None,None
27275,99991,151118,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DIVERTICULITIS/SDA,329,MS,MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC,None,None
27276,99992,197084,EMERGENCY,CLINIC REFERRAL/PREMATURE,RETROPERITONEAL HEMORRHAGE,907,MS,OTHER O.R. PROCEDURES FOR INJURIES W MCC,None,None
27277,99995,137810,ELECTIVE,PHYS REFERRAL/NORMAL DELI,ABDOMINAL AORTIC ANEURYSM/SDA,237,MS,MAJOR CARDIOVASC PROCEDURES W MCC,None,None


In [17]:
admit = pd.read_sql('''SELECT * from admissions limit 10;''',conn)
admit

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,None,None,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,None,None,BRAIN MASS,0,1
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,None,None,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,None,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,None,None,V-TACH,0,1
6,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,None,CATHOLIC,None,WHITE,None,None,NEWBORN,0,1
7,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,None,None,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,None,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,None,None,UNSTABLE ANGINA\CATH,0,1
9,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,MARRIED,WHITE,None,None,STATUS EPILEPTICUS,1,1


In [18]:
data_los = pd.read_sql("""
                   SELECT distinct
                          adm.subject_id, 
                          adm.hadm_id,
                          admission_type,
                          admission_location,
                          adm.diagnosis,
                          cast(JulianDay(adm.dischtime) - JulianDay(adm.admittime) as int) as LOS,
                          drg.DRG_CODE,
                          drg.DRG_TYPE,
                          drg.DESCRIPTION AS DRG_DESC,
                          drg.DRG_SEVERITY,
                          drg.DRG_MORTALITY
                    FROM admissions as adm
                    JOIN drg on drg.subject_id = adm.subject_id and drg.hadm_id = adm.hadm_id
                    WHERE DRG_TYPE = 'MS'
                    ORDER BY LOS desc, adm.subject_id, adm.hadm_id""",conn)
data_los.to_csv('../Data/data_los.csv')
data_los


,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DIAGNOSIS,LOS,DRG_CODE,DRG_TYPE,DRG_DESC,DRG_SEVERITY,DRG_MORTALITY
0,93321,115396,EMERGENCY,CLINIC REFERRAL/PREMATURE,ANEMIA,206,14,MS,ALLOGENEIC BONE MARROW TRANSPLANT,None,None
1,6702,157559,ELECTIVE,TRANSFER FROM OTHER HEALT,ASPIRATION; FAILURE TO THRIVE,191,4,MS,"TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NE...",None,None
2,29899,169327,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,EXTREME PREMATURITY,169,790,MS,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,None,None
3,43126,124079,ELECTIVE,PHYS REFERRAL/NORMAL DELI,CHRONIC LYMPHOCYTIC LEUKEMIA\BONE MARROW TRANS...,169,9,MS,None,None,None
4,3250,171462,EMERGENCY,CLINIC REFERRAL/PREMATURE,AMC;FEVER,164,9,MS,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
27274,99527,173534,EMERGENCY,CLINIC REFERRAL/PREMATURE,ILIAC VEIN THROMBOSIS,0,299,MS,PERIPHERAL VASCULAR DISORDERS W MCC,None,None
27275,99534,130623,EMERGENCY,CLINIC REFERRAL/PREMATURE,S/P PEDESTRIAN STRUCK:INTRACRANIAL HEMORRHAGE,0,87,MS,"TRAUMATIC STUPOR & COMA, COMA <1 HR W/O CC/MCC",None,None
27276,99553,164310,EMERGENCY,EMERGENCY ROOM ADMIT,CARDIAC ARREST,0,296,MS,"CARDIAC ARREST, UNEXPLAINED W MCC",None,None
27277,99756,172485,EMERGENCY,CLINIC REFERRAL/PREMATURE,SEPSIS,0,871,MS,SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W...,None,None


In [19]:
d = pd.read_sql("""select * from admissions where subject_id = '93321';""",conn)
d

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,93321,115396,2128-03-24 23:01:00,2128-10-17 09:14:00,2128-10-17 09:14:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Private,None,CATHOLIC,WIDOWED,WHITE,2128-03-24 22:29:00,2128-03-25 00:19:00,ANEMIA,1,1


In [20]:
e = pd.read_sql('''select * from notes where subject_id = '93321';''',conn)
e

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,93321,115396.0,2128-10-17,None,None,Discharge summary,Report,None,None,Admission Date: [**2128-3-24**] ...
1,93321,115396.0,2128-06-02,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Left ve...
2,93321,115396.0,2128-03-25,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Chemoth...
3,93321,115396.0,2128-08-31,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Endocar...
4,93321,115396.0,2128-08-24,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Endocar...
...,...,...,...,...,...,...,...,...,...,...
114,93321,115396.0,2128-09-10,2128-09-10 21:47:00,None,Radiology,CHEST (PORTABLE AP),None,None,[**2128-9-10**] 9:47 PM\n CHEST (PORTABLE AP);...
115,93321,115396.0,2128-09-14,2128-09-14 08:49:00,None,Radiology,CHEST (PORTABLE AP),None,None,[**2128-9-14**] 8:49 AM\n CHEST (PORTABLE AP) ...
116,93321,115396.0,2128-09-16,2128-09-16 08:04:00,None,Radiology,CHEST (PORTABLE AP),None,None,[**2128-9-16**] 8:04 AM\n CHEST (PORTABLE AP) ...
117,93321,115396.0,2128-04-13,2128-04-13 16:54:00,None,Radiology,CT ABD & PELVIS WITH CONTRAST,None,None,[**2128-4-13**] 4:54 PM\n CT ABD & PELVIS WITH...


In [21]:
val = e.TEXT.values[0]
val

'Admission Date:  [**2128-3-24**]              Discharge Date:   [**2128-10-17**]\n\nDate of Birth:  [**2066-4-14**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nNo Known Allergies / Adverse Drug Reactions\n\nAttending:[**First Name3 (LF) 2712**]\nChief Complaint:\nFatigue/Shortness of Breath\n\nMajor Surgical or Invasive Procedure:\nRight PICC placement [**2128-3-25**]\nCentral line placement [**2128-3-25**]\nBone marrow biopsy [**2128-3-25**]\nLeft subclavian central line placement [**2128-3-26**]\nBone marrow biopsy [**2128-4-8**]\nLeft PICC placement [**2128-4-10**]\nLeft internal jugular central line placement [**2128-4-20**]\nBone marrow biopsy [**2128-5-7**]\nBone marrow biopsy [**2128-5-27**]\nBone marrow biopsy [**2128-6-9**]\nBronchoscopy [**2128-6-10**]\nBone Marrow Biopsy on [**2128-6-29**]\nPercutaneous cholecystostomy tube placement [**2128-8-9**]\nPICC placement on [**2128-8-14**]\n\n\nHistory of Present Illness:\nMr. [**Known lastname 74075**] is a 61 yo M w